<a href="https://colab.research.google.com/github/iosifdaniel07/-Handwriting-to-Text-Converter-/blob/main/federaded_learning_testt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import copy
from sklearn.model_selection import train_test_split

from google.colab import files
!pip install gdown

from google.colab import files
!pip install -U --no-cache-dir gdown --pre
from google.colab import drive
drive.mount('/content/drive')
#!ls /content/drive/MyDrive
!cp "/content/drive/MyDrive/data_set_normalize_c.csv" .

# 1. Încarcă și preprocesează datele
data_set = pd.read_csv('data_set_normalize_c.csv')
data_set.replace([np.inf, -np.inf], np.nan, inplace=True)
data_set.fillna(data_set.median(), inplace=True)

cols_to_drop = [
    'ECE Flag Count',
    'RST Flag Count',
    'Fwd URG Flags',
    'CWE Flag Count',
    'Fwd PSH Flags',
    'Fwd Header Length.1',
    'Idle Std',
    'Down/Up Ratio',
    'Active Max',
    'Active Min',
    'Subflow Bwd Bytes',
    'Active Max',
    'Subflow Bwd Packets',
    'Subflow Fwd Packets',
    'Total Backward Packets',
    'Fwd IAT Std',
    'Active Mean',
    'act_data_pkt_fwd',
    'Total Length of Fwd Packets',
    'Total Fwd Packets'
]

X = data_set.drop(columns=['Label'])
X = data_set.drop(columns=cols_to_drop)
y = data_set['Label']

timesteps = 5

def create_sequences(data, labels, timesteps=5):
    X_seq, y_seq = [], []
    for i in range(len(data) - timesteps):
        X_seq.append(data[i:i + timesteps])
        y_seq.append(labels[i + timesteps - 1])
    return np.array(X_seq), np.array(y_seq)

X_seq_all, y_seq_all = create_sequences(X, y, timesteps)
X_seq_all = X_seq_all.astype(np.float32)
y_seq_all = y_seq_all.astype(np.int64)

# 2. Împarte datele în train (80%) și validare (20%)
X_train_initial, X_val, y_train_initial, y_val = train_test_split(
    X_seq_all, y_seq_all, test_size=0.2, random_state=42, stratify=y_seq_all
)

#data train big model
X_train, X_train_bm, y_train, y_train_bm = train_test_split(
    X_train_initial, y_train_initial, test_size=0.1, random_state=42, stratify=y_seq_all
)

print("X_train shape - shared models :", X_train.shape)
print("x_train shape - big model:", X_train_bm.shape)
print("x_train shape - validation:", X_val.shape)

# 3. Definește modelul
def create_keras_model_v4():
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(timesteps, X_train.shape[2]) ),
        Dropout(0.3),
        LSTM(64, return_sequences=False),
        Dropout(0.3),
        Dense(100, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss=BinaryCrossentropy(), optimizer=Nadam(), metrics=[AUC(name='auc')])
    return model

early_stopping = EarlyStopping(monitor='loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.001, patience=3, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='loss', save_best_only=True, verbose=1)

global_model = create_keras_model_v4()

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_and_plot_model(model, history, X_test, y_test, labels=['Normal', 'Anomaly']):
    """
    Evaluează modelul și afișează grafice pentru loss, auc/accuracy, clasificare și matricea de confuzie.

    Args:
        model: modelul Keras antrenat.
        history: obiectul returnat de `model.fit()`.
        X_test: datele de testare.
        y_test: etichetele reale pentru test.
        labels: etichetele de clasă (default: ['Normal', 'Anomaly']).
    """

    # -- Plot training history
    plt.figure(figsize=(14, 5))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss', color='blue')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # AUC or Accuracy
    plt.subplot(1, 2, 2)
    if 'auc' in history.history:
        plt.plot(history.history['auc'], label='Training AUC', color='green')
        if 'val_auc' in history.history:
            plt.plot(history.history['val_auc'], label='Validation AUC', color='red')
        plt.ylabel('AUC')
        plt.title('AUC over Epochs')
    elif 'accuracy' in history.history:
        plt.plot(history.history['accuracy'], label='Training Accuracy', color='green')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
        plt.ylabel('Accuracy')
        plt.title('Accuracy over Epochs')
    else:
        plt.title('No Accuracy/AUC Found')

    plt.xlabel('Epochs')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # -- Evaluate the model
    loss, metric = model.evaluate(X_test, y_test, verbose=1)
    print(f"\nTest Loss: {loss:.4f}")
    print(f"Test Metric (Accuracy/AUC): {metric:.4f}")

    # -- Predict and analyze results
    y_pred_probs = model.predict(X_test)
    y_pred_classes = (y_pred_probs > 0.5).astype(int).reshape(-1)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_classes, target_names=labels))

    # -- Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

history = global_model.fit(
    X_train_bm, y_train_bm,
    epochs=25,
    batch_size=128,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    validation_split=0.2,
    verbose=1
)

evaluate_and_plot_model(global_model, history, X_val, y_val)

import numpy as np

def save_weights_to_txt(weights, filename="global_weights.txt"):
    """
    Salvează greutățile modelului (listă de numpy arrays) într-un fișier text.

    Args:
        weights: listă de numpy arrays (ex: model.get_weights()).
        filename: numele fișierului text de ieșire.
    """
    with open(filename, "w") as f:
        for i, weight_array in enumerate(weights):
            f.write(f"--- Layer {i} (shape: {weight_array.shape}) ---\n")
            # Salvăm valorile aplatizate pentru lizibilitate
            np.savetxt(f, weight_array.reshape(-1), fmt="%.6f")
            f.write("\n\n")
    print(f"[✅] Weights saved to {filename}")


# 4. Setări FL
NUM_CLIENTS = 10
ROUNDS = 2
BATCH_SIZE = 128
split_size = len(X_train) // NUM_CLIENTS

# 5. Împarte datele de antrenare în datasets pe client
client_datasets = []
for i in range(NUM_CLIENTS):
    start = i * split_size
    end = (i + 1) * split_size
    ds = tf.data.Dataset.from_tensor_slices((X_train[start:end], y_train[start:end]))
    ds = ds.shuffle(buffer_size=128).batch(BATCH_SIZE)
    client_datasets.append(ds)
    print(f"Client {i+1} dataset shape: {ds.element_spec}")

# 6. Inițializează modelul global
global_weights = global_model.get_weights()
save_weights_to_txt

# 7. Simulează runde federate
for round_num in range(ROUNDS):
    print(f"\n--- Federated Round {round_num+1} ---")
    local_weights = []

    for client_data in client_datasets:
        client_model = create_keras_model_v4()
        client_model.set_weights(copy.deepcopy(global_weights))

        history = client_model.fit(
                          X_train_bm, y_train_bm,
                          epochs=1,
                          batch_size=128,
                          callbacks=[early_stopping, reduce_lr, checkpoint],
                          validation_split=0.2,
                          verbose=1
                        )
        evaluate_and_plot_model(global_model, history, X_test, y_test
        save_weights_to_txt(client_model.get_weights(), f"client_{len(local_weights)}.txt")
        local_weights.append(client_model.get_weights())

    # Agregare (medie simplă)
    new_weights = []
    for weights in zip(*local_weights):
        new_weights.append(np.mean(np.array(weights), axis=0))

    global_weights = new_weights
    global_model.set_weights(global_weights)

    evaluate_and_plot_model(global_model, history, X_test, y_test)

# 8. Evaluare finală pe datele de validare (cele 20%)
evaluate_and_plot_model(global_model, history, X_val, y_val)


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



--- Federated Round 1 ---
Interim Eval — Loss: 0.0063, AUC: 1.0000

--- Federated Round 2 ---
Interim Eval — Loss: 0.0010, AUC: 1.0000

--- Federated Round 3 ---
Interim Eval — Loss: 0.0011, AUC: 1.0000

--- Federated Round 4 ---
Interim Eval — Loss: 0.0010, AUC: 1.0000

--- Federated Round 5 ---
Interim Eval — Loss: 0.0013, AUC: 1.0000

✅ Final Evaluation on Validation Set — Loss: 0.0033, AUC: 0.9999
